In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import pickle
import numpy as np
import pandas as pd
import skimage.io as io
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

from keras.applications.resnet50 import preprocess_input
from keras.models import Model

### In this part, we conduct the following procedure to make our data be analytic-ready.
**Step 1.** For every species, we select out the **representative images**.

**Step 2.** For every species representative image, we calculate its **HSV values with regard of different parts** (body, forewing, hindwing, whole)

**Step 3.** For every species representative image, we extract its **2048-dimensional features** from the well-trained neural network model

**Step 4.** We cluster species based on either the 2-dimensional t-SNE map or 2048D features into **k assemblages through k-Means Clustering**

**Step 5.** We use **t-SNE to compress its 2048-dimensional features** into one dimension as the trait value

**Step 6.** We quantify the **assemblage-level color diversity** by calculating the average cosine distance among every pair of species in the same assemblage


### output files:
1. **all_complete_table.csv**: main result for further analysis where a row implies a **species**
2. **trait_analysis.csv**: trait value for T-statistics analysis (T stands for trait), where a row implies an **image**
3. **cluster_center.csv**: information about assemblage centers where a row implies an assemblage center
4. **in-cluser_pairwise_diversity.csv**: result of pair-wise color distance where a row implies a pair of species

In [2]:
model_dirname = '/home/put_data/moth/code/cmchang/regression/fullcrop_dp0_newaug-rmhue+old_species_keras_resnet_fold_20181121_4'

In [3]:
# read testing dataset and set the path to obtain every part's mask
Xtest = pd.read_csv(os.path.join(model_dirname, 'test.csv'))
Xtest['img_rmbg_path'] = Xtest.Number.apply(lambda x: '/home/put_data/moth/data/whole_crop/'+str(x)+'.png')
Xtest['img_keep_body_path'] = Xtest.img_rmbg_path.apply(lambda x: x.replace('whole_crop','KEEP_BODY'))
Xtest['img_keep_down_path'] = Xtest.img_rmbg_path.apply(lambda x: x.replace('whole_crop','KEEP_DOWN'))
Xtest['img_keep_up_path'] = Xtest.img_rmbg_path.apply(lambda x: x.replace('whole_crop','KEEP_UP'))

Xtest = Xtest.reset_index()
Xtest.drop(columns='index', inplace=True)

In [4]:
# get the dictionary to look up the average elevation of a species
with open(os.path.join('/home/put_data/moth/metadata/1121_Y_mean_dict.pickle'), 'rb') as handle:
    Y_dict = pickle.load(handle)
Ytest = np.vstack(Xtest['Species'].apply(lambda x: Y_dict[x]))

In [5]:
# aggregate the testing data by Species
df_species_group = Xtest.groupby('Species').apply(
    lambda g: pd.Series({
        'indices': g.index.tolist(),
    }))
df_species_group = df_species_group.sample(frac=1).reset_index()
display(df_species_group.head())

,Species,indices
0,Comibaena cassidara,[2022]
1,Trichoplites albimaculosa,[1597]
2,Comibaena delicatior,"[100, 1316, 3361]"
3,Ourapteryx venusta,[3266]
4,Euproctis kanshireia,"[170, 999, 1024, 1379, 1747, 2597, 2838, 2866,..."


### Step 1.

In [6]:
# select out the representative image which is the closest to its average elevation
sel = list()
for k in range(df_species_group.shape[0]):
    row = df_species_group.iloc[k]
    i = np.argmin(np.abs(np.array(Xtest.Alt[row['indices']]) - Y_dict[row['Species']]))
    sel.append(row['indices'][i])

In [7]:
# Xout: DataFrame only contains representative images
Xout = Xtest.iloc[sel]
Yout = Ytest[sel]

Xout = Xout.reset_index()
Xout.drop(columns='index', inplace=True)
Xout.head()

,Number,Family,Genus,Species,Latitude,Longititude,Alt,date,img_path,Filename,...,Alt_Q95,Alt_Q100,Alt_mean,Alt_class,srad,tavg,img_rmbg_path,img_keep_body_path,img_keep_down_path,img_keep_up_path
0,V10-20150116-051,Geometridae,Comibaena,Comibaena cassidara,23.665211,120.633595,296.0,20150116.0,/home/dirl/Desktop/moth/data/flickr/V10-201501...,V10-20150116-051.jpg,...,900.00,1000.0,398.777778,0.0,11926.0,15.500000,/home/put_data/moth/data/whole_crop/V10-201501...,/home/put_data/moth/data/KEEP_BODY/V10-2015011...,/home/put_data/moth/data/KEEP_DOWN/V10-2015011...,/home/put_data/moth/data/KEEP_UP/V10-20150116-...
1,V12-20131128-025,Geometridae,Trichoplites,Trichoplites albimaculosa,23.618729,120.928362,2300.0,20131128.0,/home/dirl/Desktop/moth/data/flickr/V12-201311...,V12-20131128-025.jpg,...,3002.00,3002.0,2859.548387,5.0,12090.0,9.000000,/home/put_data/moth/data/whole_crop/V12-201311...,/home/put_data/moth/data/KEEP_BODY/V12-2013112...,/home/put_data/moth/data/KEEP_DOWN/V12-2013112...,/home/put_data/moth/data/KEEP_UP/V12-20131128-...
2,A34-20140626-064,Geometridae,Comibaena,Comibaena delicatior,24.815020,121.412850,670.0,20140626.0,/home/dirl/Desktop/moth/data/flickr/A34-201406...,A34-20140626-064.jpg,...,3002.00,3002.0,1920.375000,3.0,18227.0,22.500000,/home/put_data/moth/data/whole_crop/A34-201406...,/home/put_data/moth/data/KEEP_BODY/A34-2014062...,/home/put_data/moth/data/KEEP_DOWN/A34-2014062...,/home/put_data/moth/data/KEEP_UP/A34-20140626-...
3,V04-20140721-152,Geometridae,Ourapteryx,Ourapteryx venusta,24.257227,121.008787,2287.0,20140721.0,/home/dirl/Desktop/moth/data/flickr/V04-201407...,V04-20140721-152.jpg,...,3002.00,3002.0,2497.777778,4.0,20143.0,16.200001,/home/put_data/moth/data/whole_crop/V04-201407...,/home/put_data/moth/data/KEEP_BODY/V04-2014072...,/home/put_data/moth/data/KEEP_DOWN/V04-2014072...,/home/put_data/moth/data/KEEP_UP/V04-20140721-...
4,V02-20140303-079,Erebidae,Euproctis,Euproctis kanshireia,24.051142,120.780668,312.0,20140303.0,/home/dirl/Desktop/moth/data/flickr/V02-201403...,V02-20140303-079.jpg,...,1072.25,1220.0,347.263158,0.0,14704.0,18.200001,/home/put_data/moth/data/whole_crop/V02-201403...,/home/put_data/moth/data/KEEP_BODY/V02-2014030...,/home/put_data/moth/data/KEEP_DOWN/V02-2014030...,/home/put_data/moth/data/KEEP_UP/V02-20140303-...


### Step 2. 

In [8]:
# extract the HSV features for species representatives
import skimage.color as color
def img_metrics(img):
    hsv = color.rgb2hsv(img)
    mask = 1.0 - (np.mean(img, axis=2)==255.0) + 0.0
    x,y = np.where(mask)
    mean_hsv = np.mean(hsv[x,y], axis=0)
    std_hsv = np.std(hsv[x,y], axis=0)
    return mean_hsv, std_hsv

df_reg_list = list()
species_list = list()
filename_list = list()
for k in range(Xout.shape[0]):
    print(k, end='\r')
    species = Xout.iloc[k]['Species']
    species_list.append(species)
    
    body_img = io.imread(Xout.iloc[k]['img_keep_body_path'])
    mask = 1.0 - (np.mean(body_img, axis=2)==255.0) + 0.0
    body_img[:,:,0] = body_img[:,:,0]*mask
    body_img[:,:,1] = body_img[:,:,1]*mask
    body_img[:,:,2] = body_img[:,:,2]*mask

    img = io.imread(Xout.iloc[k]['img_keep_up_path'])

    img += body_img

    alt = Y_dict[Xout.iloc[k]['Species']]
    
    mean_hsv, std_hsv = img_metrics(img)
    
    whole_img = io.imread(Xout.iloc[k]['img_rmbg_path'])
    whole_mean_hsv, whole_std_hsv = img_metrics(whole_img)
    
    res = np.append(whole_mean_hsv[:3], mean_hsv[:3])
    res = np.append(res, [alt])
    
    df_reg_list.append(res)
    
df_reg_output = pd.DataFrame(data=df_reg_list,
                             columns=['h.whole', 's.whole', 'v.whole', 
                                      'h.body_fore','s.body_fore', 'v.body_fore','alt'])

### Step 3. 

In [9]:
# extract 2048-dimensional features
from keras.models import load_model
model = load_model(os.path.join(model_dirname,'model.h5'))
features = model.get_layer('global_average_pooling2d_1')
extractor = Model(inputs=model.input, outputs=features.output)

TestImg = list()
for i in range(Xout.shape[0]):
    img = io.imread(list(Xout['img_rmbg_path'])[i])
    TestImg.append(img)
TestImg = np.stack(TestImg)
TestInput = preprocess_input(TestImg.astype(float))

Fout = extractor.predict(x=TestInput)
Yout = np.array([Y_dict[sp] for sp in Xout.Species])

In [10]:
np.save(file='Species_Representative_1047x2048.npy', arr=Fout)

In [11]:
Fout.shape

(1047, 2048)

### Step 4.

In [13]:
# compress 2048-D features to 2-D map for visualization and clustering
from sklearn.manifold import TSNE
F_embedded = TSNE(n_components=2, perplexity=120).fit_transform(Fout)

In [54]:
from sklearn.cluster import KMeans
from sklearn import metrics
from time import time

def bench_k_means(estimator, name, data):
    t0 = time()
    estimator.fit(data)
    print('%-9s\t%.2fs\t%.3f\t%.3f'
          % (name, (time() - t0), estimator.inertia_,
             metrics.silhouette_score(data, estimator.labels_,
                                      metric='cosine',
                                      sample_size=500)))
    return estimator
for k in [30]:
    km = KMeans(init='k-means++', n_clusters=k, n_init=20)
    km = bench_k_means(km, name="k-means++", data=Fout)


k-means++	7.64s	64836.930	0.054


In [55]:
from collections import Counter
Counter(km.labels_)

Counter({0: 49,
         1: 86,
         2: 65,
         3: 13,
         4: 1,
         5: 5,
         6: 24,
         7: 1,
         8: 6,
         9: 88,
         10: 93,
         11: 3,
         12: 5,
         13: 92,
         14: 40,
         15: 36,
         16: 61,
         17: 49,
         18: 70,
         19: 1,
         20: 13,
         21: 41,
         22: 10,
         23: 33,
         24: 56,
         25: 47,
         26: 2,
         27: 39,
         28: 15,
         29: 3})

In [56]:
Xout['tsne.0'] = F_embedded[:,0]
Xout['tsne.1'] = F_embedded[:,1]
Xout['km_label'] = km.labels_

In [57]:
# representative image information
resout = pd.concat([Xout, df_reg_output], axis=1)
resout.to_csv(os.path.join(model_dirname, 'all_complete_table.csv'), index=False)

#### - If clustering based on t-SNE maps

In [39]:
# # cluster information
# stat = Xout[['km_label','Alt']].groupby('km_label').apply(np.mean)
# stat = stat.sort_values('Alt')
# stat.columns = ['km_label', 'class_alt']

# # center information
# centers = km.cluster_centers_
# myk = km.cluster_centers_.shape[0]
# centx, centy = list(), list()
# for i in range(stat.shape[0]):
#     centx.append(centers[int(stat.iloc[i]['km_label']),0])
#     centy.append(centers[int(stat.iloc[i]['km_label']),1])

# # add center information into clustere information
# stat['center_x'] = centx
# stat['center_y'] = centy
# stat['order'] = np.arange(myk)

# # output cluster information
# stat.to_csv(os.path.join(model_dirname,'cluster_center.csv'), index=False)

#### - If clustering based on 2048D features

In [58]:
from sklearn.metrics.pairwise import pairwise_distances

# cluster information
stat = Xout[['km_label','Alt']].groupby('km_label').apply(np.mean)
stat = stat.sort_values('km_label')
stat.columns = ['km_label', 'class_alt']

# center information
centers = km.cluster_centers_
myk = km.cluster_centers_.shape[0]
centx, centy = list(), list()

for i in range(myk):
    center = centers[i:(i+1),:]
    sel = np.where(km.labels_==i)[0]
    nearest_species = np.argmin(pairwise_distances(X=center, Y=Fout2[sel], metric='cosine'))
    i_nearest_species = sel[nearest_species]
    centx.append(F_embedded[i_nearest_species, 0])
    centy.append(F_embedded[i_nearest_species, 1])

# add center information into clustere information
stat['center_x'] = centx
stat['center_y'] = centy

stat = stat.sort_values('class_alt')
# stat.columns = ['km_label', 'class_alt']
stat['order'] = np.arange(myk)

# output cluster information
stat.to_csv(os.path.join(model_dirname,'cluster_center.csv'), index=False)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: FutureWarning: 'km_label' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  """


### Step 5.

In [22]:
# compress 2048-D features to 1-D trait for functional trait analysis
TestImg = list()
for i in range(Xtest.shape[0]):
    img = io.imread(list(Xtest['img_rmbg_path'])[i])
    TestImg.append(img)
TestImg = np.stack(TestImg)
TestInput = preprocess_input(TestImg.astype(float))
Ftest = extractor.predict(x=TestInput)

In [ ]:
from sklearn.manifold import TSNE
F_trait = TSNE(n_components=1, perplexity=100).fit_transform(Ftest)
F_trait = F_trait - np.min(F_trait)

Xtest['trait'] = F_trait[:,0]

In [23]:
np.save(file='Species_TestingInstance_4249x2048.npy', arr=Ftest)

In [59]:
# image trait information table
dtrait = pd.merge(Xtest[['Species', 'trait']], resout[['Species','km_label','alt']], how='left', on='Species')
dtrait.to_csv(os.path.join(model_dirname, 'trait_analysis.csv'), index=False)

### Step 6.

In [60]:
# calculate in-cluster pairwise distance
from sklearn.metrics.pairwise import pairwise_distances

# just convert the cluster labels to be ordered for better visualization in the next analysis
km_label_to_order = dict()
order_to_km_label = dict()
for i in range(myk):
    km_label_to_order[int(stat.iloc[i]['km_label'])] = i
    order_to_km_label[i] = int(stat.iloc[i]['km_label'])

pair_diversity = np.array([])
order = np.array([])
for k in range(myk):
    
    this_km_label = order_to_km_label[k] 
    sel = np.where(resout.km_label == this_km_label)[0]
    if len(sel) == 1:
        t = np.array([[0]])
        dist_list = np.array([0])
    else:
        t = pairwise_distances(Fout[sel, :], metric='cosine')
        dist_list = np.array([])
        for i in range(t.shape[0]):
            dist_list = np.append(dist_list,t[i,(i+1):])
    pair_diversity = np.append(pair_diversity, dist_list)
    order = np.append(order, np.repeat(k, len(dist_list)))
    
di = pd.DataFrame({'diversity': pair_diversity,
              'order': order})
di.to_csv(os.path.join(model_dirname, 'in-cluser_pairwise_diversity.csv'), index=False)